In [1]:
!pip install gensim
#It extracts semantics topics from documents


  Using cached Cython-0.29.23-cp39-cp39-win_amd64.whl (1.7 MB)
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.28
    Uninstalling Cython-0.29.28:
      Successfully uninstalled Cython-0.29.28


In [2]:
!pip3 install spacy
!pip3 install pyldavis
#import pyLDAvis.genism_models
#pyLDAvis.enable_notebook()
import pandas as pd
import numpy as np
# spacy for lemmatization
import spacy

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim 
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
%matplotlib inline


import string
import re
from pprint import pprint

# NLTK 
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = stopwords.words('english') #this depends on each language

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel



# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\Harshitha\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
df_depression = pd.read_csv('depression_posts.csv')
print(df_depression.shape)
df_depression.head(3)

(3903, 13)


,title,score,p_id,subreddit,url,num_comments,body,p_timestamp,c_id,comment,c_timestamp,Post_Reply,Time_to_Comment
0,Our most-broken and least-understood rules is ...,2361,doqwow,depression,https://www.reddit.com/r/depression/comments/d...,180,We understand that most people who reply immed...,2019-10-29 20:22:02,f5pot56,Understood and I apologise if I forget in the ...,2019-10-29 21:23:38,Y,0 days 01:01:36
1,Our most-broken and least-understood rules is ...,2361,doqwow,depression,https://www.reddit.com/r/depression/comments/d...,180,We understand that most people who reply immed...,2019-10-29 20:22:02,f5pot7j,I agree wholeheartedly. When you're strugglin...,2019-10-29 21:23:39,Y,0 days 01:01:37
2,Our most-broken and least-understood rules is ...,2361,doqwow,depression,https://www.reddit.com/r/depression/comments/d...,180,We understand that most people who reply immed...,2019-10-29 20:22:02,f647tsy,Biggest Problem on private talks may be that y...,2019-11-02 15:41:35,Y,3 days 19:19:33


In [4]:
data=df_depression['comment'].values.tolist()

In [5]:
#tokenize and text cleanup
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['understood', 'and', 'apologise', 'if', 'forget', 'in', 'the', 'future', 'and', 'break', 'it']]


In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['understood', 'and', 'apologise', 'if', 'forget', 'in', 'the', 'future', 'and', 'break', 'it']


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

## Let's call the function in order

In [8]:

#To run this code it is necessary to get the spacy moidel: "!python - m spacy download en", last depends on the used language 

# Remove Stop Words
#!python - m spacy download en
!python -m spacy download en_core_web_sm
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'es' model, keeping only tagger component (for efficiency)
!python3 -m spacy download es
nlp = spacy.load("en_core_web_sm")
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.


[['understand', 'apologise', 'forget', 'future', 'break']]


In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [10]:
print(id2word[0])
print(id2word[5])
print(id2word[2])
print(id2word[7])
print(id2word[1])
print(id2word[20])

apologise
accessible
forget
afraid
break
contribution


In [11]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('apologise', 1),
  ('break', 1),
  ('forget', 1),
  ('future', 1),
  ('understand', 1)],
 [('accessible', 1),
  ('acquaint', 1),
  ('afraid', 1),
  ('agree', 1),
  ('anywhere', 1),
  ('apply', 1),
  ('aside', 1),
  ('base', 1),
  ('bit', 1),
  ('change', 1),
  ('come', 2),
  ('comfort', 1),
  ('comment', 1),
  ('confidence', 1),
  ('contribute', 1),
  ('contribution', 1),
  ('contributor', 1),
  ('creation', 2),
  ('damage', 1),
  ('difficult', 1),
  ('direct', 1),
  ('disagree', 1),
  ('discussion', 3),
  ('dislike', 1),
  ('else', 1),
  ('even', 2),
  ('exception', 1),
  ('feel', 1),
  ('generally', 1),
  ('get', 2),
  ('group', 1),
  ('honest', 1),
  ('important', 1),
  ('interaction', 1),
  ('interesting', 1),
  ('internet', 1),
  ('issue', 3),
  ('keep', 1),
  ('lack', 1),
  ('leave', 1),
  ('look', 1),
  ('lot', 1),
  ('low', 1),
  ('lurker', 1),
  ('majority', 1),
  ('make', 5),
  ('much', 2),
  ('new', 1),
  ('number', 1),
  ('open', 2),
  ('part', 1),
  ('peak', 1),
  ('peo

In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=4,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.052*"change" + 0.045*"back" + 0.042*"therapy" + 0.036*"experience" + '
  '0.034*"seem" + 0.030*"person" + 0.025*"put" + 0.024*"able" + 0.022*"maybe" '
  '+ 0.022*"enough"'),
 (1,
  '0.147*"money" + 0.077*"world" + 0.053*"easy" + 0.043*"pretty" + '
  '0.043*"treat" + 0.033*"truly" + 0.032*"exactly" + 0.032*"beautiful" + '
  '0.032*"human" + 0.029*"true"'),
 (2,
  '0.291*"thank" + 0.084*"reach" + 0.067*"whole" + 0.035*"free" + 0.032*"safe" '
  '+ 0.024*"sub" + 0.022*"develop" + 0.017*"clear" + 0.009*"misery" + '
  '0.007*"regard"'),
 (3,
  '0.052*"feel" + 0.035*"know" + 0.033*"get" + 0.030*"thing" + 0.029*"make" + '
  '0.028*"go" + 0.023*"well" + 0.022*"life" + 0.022*"time" + 0.022*"good"'),
 (4,
  '0.065*"big" + 0.058*"option" + 0.055*"case" + 0.048*"include" + '
  '0.041*"chance" + 0.035*"deep" + 0.034*"progress" + 0.030*"share" + '
  '0.029*"useless" + 0.022*"drop"'),
 (5,
  '0.098*"word" + 0.078*"kind" + 0.064*"push" + 0.048*"possible" + '
  '0.047*"appreciate" + 0.046*"ins

In [14]:
# Compute Perplexity

print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.950990755787092

Coherence Score:  0.4132638751252635


In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\Harshitha\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.448534  0.008856       1        1  56.488349
10    -0.334850 -0.238982       2        1  14.627898
0     -0.272596  0.310497       3        1   8.973566
1     -0.029735 -0.119397       4        1   3.283783
8     -0.001217 -0.003269       5        1   2.701903
17     0.000907 -0.025137       6        1   2.606621
5      0.015075  0.004729       7        1   2.186728
4      0.049880  0.005307       8        1   1.714723
18     0.062710  0.005973       9        1   1.263311
16     0.061569  0.005752      10        1   1.231858
2      0.064366  0.005856      11        1   0.970114
11     0.081716  0.005549      12        1   0.924184
19     0.083485  0.005332      13        1   0.621260
14     0.090610  0.004959      14        1   0.620949
7      0.084854  0.005258      15        1   0.607528
15     0.089259  0.005085      16        1   0.500841
9      0.098216  0.004102      17        1   0.300345
12     0.101052  0.003472      18        1   0.169979
13     0.100628  0.003554      19        1   0.161054
6      0.102606  0.002504      20        1   0.045008, topic_info=           Term        Freq       Total Category  logprob  loglift
92         work  483.000000  483.000000  Default  30.0000  30.0000
399       money  354.000000  354.000000  Default  29.0000  29.0000
137  depression  712.000000  712.000000  Default  28.0000  28.0000
198       thank  207.000000  207.000000  Default  27.0000  27.0000
285         way  696.000000  696.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
25       direct    0.005421    3.425996  Topic20  -8.7124   1.2573
26     disagree    0.005421    1.047739  Topic20  -8.7124   2.4420
27   discussion    0.005421    1.047737  Topic20  -8.7124   2.4420
28      dislike    0.005421    8.943534  Topic20  -8.7124   0.2977
29         else    0.005421  185.766105  Topic20  -8.7124  -2.7358

[841 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
305       3  0.991683        able
566      12  0.982974  absolutely
841      15  0.952907      accept
568       4  0.975597         act
906       7  0.885482      active
...     ...       ...         ...
439       5  0.982411       wrong
286       1  0.998909        year
900       9  0.584809   years_ago
515       3  0.988033         yet
565       2  0.994339       young

[481 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 11, 1, 2, 9, 18, 6, 5, 19, 17, 3, 12, 20, 15, 8, 16, 10, 13, 14, 7])